In [1]:
!wget -c  https://dlcdn.apache.org/kafka/3.8.0/kafka_2.13-3.8.0.tgz

StatementMeta(, 8ba52f58-3c67-432a-856e-657f4d5b0ae0, 3, Finished, Available, Finished)

--2024-11-07 17:53:25--  https://dlcdn.apache.org/kafka/3.8.0/kafka_2.13-3.8.0.tgz
Resolving dlcdn.apache.org... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120735482 (115M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.8.0.tgz’

kafka_2.13-3.8.0.tg 100%[===================>] 115.14M   453MB/s    in 0.3s    

2024-11-07 17:53:28 (453 MB/s) - ‘kafka_2.13-3.8.0.tgz’ saved [120735482/120735482]



In [2]:
!tar -xzf kafka_2.13-3.8.0.tgz
!mv kafka_2.13-3.8.0/ kafka

StatementMeta(, 8ba52f58-3c67-432a-856e-657f4d5b0ae0, 4, Finished, Available, Finished)

In [3]:
!./kafka/bin/zookeeper-server-start.sh -daemon ./kafka/config/zookeeper.properties
!./kafka/bin/kafka-server-start.sh -daemon ./kafka/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

StatementMeta(, 8ba52f58-3c67-432a-856e-657f4d5b0ae0, 5, Finished, Available, Finished)

Waiting for 10 secs until kafka and zookeeper services are up and running


In [4]:
!pip install confluent-kafka
!pip install kafka-python

StatementMeta(, 8ba52f58-3c67-432a-856e-657f4d5b0ae0, 6, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 69.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
import socket
import time
import json
from datetime import datetime
from confluent_kafka import Producer

# Configuración de los parámetros para conectarte a Kafka en Azure Event Hubs
conf = {
    'bootstrap.servers': '',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism': 'PLAIN',
    'sasl.username': '$ConnectionString',
    'sasl.password': '',
}

# Crear el productor de Kafka
producer = Producer(**conf)

# Definir la función callback para confirmar la entrega de los mensajes
def delivery_report(err, msg):
    if err is not None:
        print(f"Mensaje fallido: {err}")
    else:
        print(f"Mensaje enviado a {msg.topic()} [{msg.partition()}] @ offset {msg.offset()}")

# Crear un socket TCP/IP
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Conectar el socket al host y puerto proporcionados
s.connect(('rtd.hpwren.ucsd.edu', 12020))

# Configuración de la tasa de envío de mensajes (10 mensajes/minuto = 1 mensaje/6 segundos)
tiempo_entre_mensajes = 6  # segundos
buffer = ""  # Buffer para acumular datos

try:
    # Bucle principal para recibir y procesar datos en tiempo real
    while True:
        # Recibir datos del socket en bloques de hasta 1024 bytes
        data = s.recv(1024).decode()  # Decodificar los datos binarios en un string legible
        
        # Dividir los datos recibidos por líneas
        for line in data.splitlines():
            # Verificar que la línea tiene el formato esperado con 4 elementos
            parts = line.split("\t")
            if len(parts) == 4:
                ip, id, tstamp, values = parts

                try:
                    # Convertir el timestamp a un objeto datetime
                    timestamp_seconds = int(tstamp)
                    tstamp_date = datetime.fromtimestamp(timestamp_seconds).strftime('%Y-%m-%d %H:%M:%S')

                    # Procesar los valores de las variables
                    for val in values.split(",")[1:]:
                        variable, measurement = val.strip().split("=")  # Crear un diccionario con los datos para enviar
                        data_dict = {
                            "IP": ip,
                            "ID": id,
                            "Variable": variable,
                            "Measurement": measurement,
                            "Timestamp": tstamp_date
                        }

                        # Convertir el diccionario en una cadena JSON
                        data_json = json.dumps(data_dict)

                        # Enviar los datos a un tópico de Kafka
                        topic =''
                        producer.produce(topic, value=data_json.encode('utf-8'), callback=delivery_report)

                        # Llamar al flush para asegurarse de que se envíen los mensajes
                        producer.poll(0)

                        print(f"Se ha enviado el mensaje: {data_json}")

                        # Pausar para mantener la tasa de un mensaje cada 6 segundos
                        time.sleep(tiempo_entre_mensajes)

                except Exception as e:
                    print(f"Ocurrió un error procesando la línea: {line}")
                    print(f"Error: {e}")
            else:
                print(f"Línea malformada o incompleta: {line}")

except KeyboardInterrupt:
    # Manejar la interrupción del teclado (Ctrl+C)
    print("Deteniendo el script...")

except Exception as e:
    # Capturar y mostrar otros tipos de excepciones que puedan ocurrir
    print("Ocurrió un error")
    print(f"Error: {e}")

finally:
    # Cerrar el socket al finalizar la ejecución del script
    s.close()
    # Vaciar el buffer de mensajes pendientes
    producer.flush()


StatementMeta(, 8ba52f58-3c67-432a-856e-657f4d5b0ae0, 7, Submitted, Running, Running)

Se ha enviado el mensaje: {"IP": "198.202.124.3", "ID": "HPWREN:LP-WXT536:0R1:4:0", "Variable": "Dn", "Measurement": "064D", "Timestamp": "2024-11-07 17:54:00"}
Mensaje enviado a es_ec8b8327-4f64-40d6-a517-fb377b181e9d [2] @ offset 160886
Se ha enviado el mensaje: {"IP": "198.202.124.3", "ID": "HPWREN:LP-WXT536:0R1:4:0", "Variable": "Dm", "Measurement": "076D", "Timestamp": "2024-11-07 17:54:00"}
Mensaje enviado a es_ec8b8327-4f64-40d6-a517-fb377b181e9d [3] @ offset 162814
Se ha enviado el mensaje: {"IP": "198.202.124.3", "ID": "HPWREN:LP-WXT536:0R1:4:0", "Variable": "Dx", "Measurement": "104D", "Timestamp": "2024-11-07 17:54:00"}
Mensaje enviado a es_ec8b8327-4f64-40d6-a517-fb377b181e9d [3] @ offset 162815
Se ha enviado el mensaje: {"IP": "198.202.124.3", "ID": "HPWREN:LP-WXT536:0R1:4:0", "Variable": "Sn", "Measurement": "5.2M", "Timestamp": "2024-11-07 17:54:00"}
Mensaje enviado a es_ec8b8327-4f64-40d6-a517-fb377b181e9d [0] @ offset 162512
Se ha enviado el mensaje: {"IP": "198.202.12